# 草稿

In [4]:
import os

from numcodecs.registry import entries
from torchvision import transforms

In [2]:
help(transforms)

Help on package torchvision.transforms in torchvision:

NAME
    torchvision.transforms

PACKAGE CONTENTS
    _functional_pil
    _functional_tensor
    _functional_video
    _presets
    _transforms_video
    autoaugment
    functional
    transforms
    v2 (package)

FILE
    c:\users\cavendishes\.conda\envs\tahoe-cvae\lib\site-packages\torchvision\transforms\__init__.py




In [6]:
help(os.walk)

Help on function walk in module os:

walk(top, topdown=True, onerror=None, followlinks=False)
    Directory tree generator.
    
    For each directory in the directory tree rooted at top (including top
    itself, but excluding '.' and '..'), yields a 3-tuple
    
        dirpath, dirnames, filenames
    
    dirpath is a string, the path to the directory.  dirnames is a list of
    the names of the subdirectories in dirpath (including symlinks to directories,
    and excluding '.' and '..').
    filenames is a list of the names of the non-directory files in dirpath.
    Note that the names in the lists are just names, with no path components.
    To get a full path (which begins with top) to a file or directory in
    dirpath, do os.path.join(dirpath, name).
    
    If optional arg 'topdown' is true or not specified, the triple for a
    directory is generated before the triples for any of its subdirectories
    (directories are generated top down).  If topdown is false, the triple


# Dataset 练手

In [23]:
from torch.utils.data import Dataset
from PIL import Image
from os import listdir
import os
from os.path import isdir,join,splitext,basename,exists

IMG_EXTENSIONS = (
    ".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"
)

def _pil_loader(path):
    with Image.open(path) as img:
        return img.convert("RGB")

def _is_image_file(filename,extensions):
    return filename.lower().endswith(tuple(ext.lower() for ext in extensions))


### ImageFolder 风格目录结构的数据集

In [24]:
def _find_classes(directory):
        """
        从指定文件夹中读取所有的类别
        :param directory:
        :return:
        """
        classes = [class_dir for class_dir in listdir(directory) if isdir(join(directory,class_dir))]
        classes.sort()
        if len(classes)==0:
            raise RuntimeError(f"目录{directory}中找不到任何类别子目录")
        return classes

class ClassDirectoryDataset(Dataset):
    """
    适用于 ImageFolder 风格目录结构的数据集，例如：

    root/
        class_a/
            img1.jpg
            ...
        class_b/
            img2.jpg
            ...

    - 自动按子文件夹名建立 `class_to_idx`
    - 收集 (image_path, label_idx) 样本列表
    - 使用给定 `transform` 和 `target_transform`
    """
    def __init__(self,root,extensions,loader=_pil_loader):
        self.root=root
        self.extensions=tuple(ext.lower() for ext in extensions)
        self.loader=loader

        self.classes=_find_classes(self.root)
        self.class_to_idx = {class_name : idx for idx,class_name in enumerate(self.classes)}
        self.samples = self._make_dataset(self.root)

    def _make_dataset(self,directory):
        instances = []
        for class_name in sorted(self.classes):
            class_dir = join(directory,class_name)
            if not isdir(class_dir):
                continue
            for root_dir,_,file_names in os.walk(class_dir):
                for file_name in sorted(file_names):
                    if _is_image_file(file_name,self.extensions):
                        path = join(root_dir,file_name)
                        item = (path,self.class_to_idx[class_name]) # 一个item是 (地址 , 类别id)的元组
                        instances.append(item)
        return instances

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        """
        根据id，返回真实的数据
        :param index:
        :return:
        """
        path,class_idx = self.samples[index]
        img_rgb=self.loader(path)
        return img_rgb,class_idx

In [25]:
myClassDirectoryDataset = ClassDirectoryDataset("../data/hymenoptera_data/train",["jpg"])
img0,lable = myClassDirectoryDataset[5]

In [26]:
img0.show()
myClassDirectoryDataset.classes[lable]

'ants'

### “图像与标签分开”的目录结构

In [27]:
def _label_path_for(image_path, label_dir):
    stem,_ = splitext(basename(image_path))
    return join(label_dir,f"{stem}.txt")


class SplitFolderLabeledDataset(Dataset):
    """
    适用于如下“图像与标签分开”的目录结构（以二分类 ants/bees 为例）：

    root/
        ants_image/
            001.jpg, 002.jpg, ...
        ants_label/
            001.txt, 002.txt, ...
        bees_image/
            101.jpg, 102.jpg, ...
        bees_label/
            101.txt, 102.txt, ...

    要求每个类有成对的 `*_image` 与 `*_label` 目录，标签文件名需与图像同名（仅扩展名不同）。
    """
    def __init__(self,root,extensions,loader=_pil_loader):
        self.root=root
        self.extensions=tuple(ext.lower() for ext in extensions)
        self.loader=loader

        self.classes = ["ants", "bees"]
        self.class_to_idx = {cls:idx for idx,cls in enumerate(self.classes)}
        self.samples= self._make_dataset()

    def _parse_label_from_file(self,label_path):
        fallback = -1
        if not exists(label_path):
            return fallback
        with open(label_path,"r",encoding="utf-8") as f:
            content = f.read().strip()

        if content in self.classes:
            return self.class_to_idx[content]
        return fallback

    def _make_dataset(self):
        instances = []
        for class_name in self.classes:
            label_dir = join(self.root,f"{class_name}_label")
            image_dir = join(self.root,f"{class_name}_image")
            if not (exists(label_dir) and exists(image_dir)):
                continue
            for file_name in listdir(image_dir):
                if not _is_image_file(file_name,self.extensions):
                    continue
                image_path = join(image_dir,file_name)
                label_path = _label_path_for(image_path,label_dir)
                target = self._parse_label_from_file(label_path)
                item = (image_path,target)
                instances.append(item)
        return instances

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        image_path,target = self.samples[index]
        img = self.loader(image_path)
        return img,target




In [28]:
mySplitFolderLabeledDataset = SplitFolderLabeledDataset("../data/practice_data/train",["jpg"])
img1,label1 = mySplitFolderLabeledDataset[200]

In [29]:
img1.show()
mySplitFolderLabeledDataset.classes[label1]

'bees'

In [30]:
from torchvision import transforms

# 可选：提供一个默认的数据增强/预处理示例，方便直接使用
def build_default_transforms(image_size=224, is_train=True):
    if is_train:
        return transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])